In [ ]:
import os
import pandas as pd
import glob
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook

In [ ]:
def read_files(root_dir):
    files = glob.glob(os.path.join(root_dir, "*.csv"))
    data_frames = {}
    for idx, file in enumerate(files):
        with open(file, 'r', encoding='utf-8') as f:
            line1 = f.readline()            
            # Example line:
            # "Sitzungsdetail für 'Bodystyling mit Bibi':"
            course_name = f.readline().split("'")[1] # get the name of the course, e.g.: Bodystyling mit Bibi        
        df = pd.read_csv(file, sep="\t", skiprows=2)
        data_frames[idx] = (df, Path(file).stem.split('_')[0], course_name)
    return data_frames

In [ ]:
def consolidate(df):
    df_cons = pd.DataFrame(columns = ['Name', 'E-Mail', 'Datum'])
    date = None
    groups = df.groupby('Name')
    for k,g in groups:
        for i, row in g.iterrows():
            df_cons = df_cons.append({'Name':row['Name'], 'E-Mail':row['E-Mail'], 'Datum':row['Datum']}, ignore_index=True)
            if date is None:
                date = row['Datum']
            break
    return (df_cons, date)

In [ ]:
def save(df, path):
    df.to_csv(path,header=True, index=False, sep='\t', encoding='utf-8')

In [ ]:
root_dir = r'C:\Users\rudi\Documents\Tina\WebexKurse\15.3-25.3'
data_frames = read_files(root_dir)
consolidated_data = []
for k,v in data_frames.items():
    df = v[0]
    name = v[1]
    course_name = v[2]
    (df_cons, date) = consolidate(df)
    consolidated_data.append((df_cons, date, course_name))
    #filename = os.path.join(root_dir, f'{name}_{date}.tsv')
    #save(df_cons, filename)

# Render Course Information into an Excel Workbook

In [ ]:
def create_excel(filename, dataframes, sheet_name='Sheet1'):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    workbook = writer.book
    header_format = workbook.add_format({'bg_color': '#689BCA', 'bold': True})
    start = 1
    
    for (df_cons, date, course_name) in dataframes:        
        
        # df1.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
        # Convert the dataframe to an XlsxWriter Excel object.
        print(f'{course_name}_{date}')
        
        df_cons.to_excel(writer, sheet_name=sheet_name, index=False, startrow=start)
        worksheet = writer.sheets[sheet_name]
        worksheet.write(start-1, 0, f'{course_name}_{date}', header_format)
        start += len(df_cons) + 4

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    writer.close()

In [ ]:
print(consolidated_data[0][0].head())

In [ ]:
create_excel('Kurse_15.03_25.03.2021.xlsx', consolidated_data)